In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import datetime
import re

from dateutil.relativedelta import relativedelta
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae

import folium
from folium import plugins
import json
import geopandas as gpd
from geopandas import gpd
from haversine import haversine
from shapely.geometry import Point, shape, mapping,Polygon
from shapely.ops import unary_union

In [2]:
censor = gpd.read_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/censor.shp')
censor

,serial,date,fp,gu,juso,lat,lon,subway,bus,building_c,building_a,pop,geometry
0,"2,992.0000",2021-01-04,"6,594.7167",???,????? ??? ??? 733-28,37.4895,126.8848,"15,223.5000","1,248.3000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
1,"2,992.0000",2021-01-05,"6,290.3333",???,????? ??? ??? 733-28,37.4895,126.8848,"14,932.0000","1,239.3000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
2,"2,992.0000",2021-01-06,"6,222.3667",???,????? ??? ??? 733-28,37.4895,126.8848,"15,729.0000","1,254.0000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
3,"2,992.0000",2021-01-07,"5,919.0333",???,????? ??? ??? 733-28,37.4895,126.8848,"16,100.5000","1,319.9000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
4,"2,992.0000",2021-01-08,"5,982.3000",???,????? ??? ??? 733-28,37.4895,126.8848,"14,750.0000","1,207.8000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26314,"4,050.0000",2021-10-19,69.1000,????,????? ???? ??? 101,37.5729,126.9556,"7,098.0000",357.6875,11,"67,916.9400","3,152.0000",POINT (37.573 126.956)
26315,"4,050.0000",2021-10-20,71.4667,????,????? ???? ??? 101,37.5729,126.9556,"9,299.0000",411.1875,11,"67,916.9400","3,152.0000",POINT (37.573 126.956)
26316,"4,050.0000",2021-10-21,67.8667,????,????? ???? ??? 101,37.5729,126.9556,"7,461.0000",357.6250,11,"67,916.9400","3,152.0000",POINT (37.573 126.956)
26317,"4,050.0000",2021-10-22,67.0000,????,????? ???? ??? 101,37.5729,126.9556,"7,634.0000",371.1250,11,"67,916.9400","3,152.0000",POINT (37.573 126.956)


In [3]:
censor['month'] = list(map(lambda x: x[5:7], censor.date))

In [4]:
censor.index = pd.to_datetime(censor.date)

In [5]:
censor['week'] = censor.date.to_period('W').index

In [6]:
sensor_wk = censor.groupby(['serial','week']).mean().reset_index()

In [7]:
sensor_wk['week2'] = list(map(lambda x: str(x)[:10], sensor_wk.week))

In [8]:
sensor_wk.week = sensor_wk.week.astype(str)

In [9]:
censor_g = censor[['serial','geometry']].drop_duplicates().reset_index(drop=True)

In [10]:
sensor_wk = pd.merge(sensor_wk, censor_g, on = 'serial',how = 'left')

In [13]:
sensor_wk.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3906 entries, 0 to 3905
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   serial      3906 non-null   float64 
 1   week        3906 non-null   object  
 2   fp          3906 non-null   float64 
 3   lat         3906 non-null   float64 
 4   lon         3906 non-null   float64 
 5   subway      3906 non-null   float64 
 6   bus         3906 non-null   float64 
 7   building_c  3906 non-null   float64 
 8   building_a  3906 non-null   float64 
 9   pop         3906 non-null   float64 
 10  week2       3906 non-null   object  
 11  geometry    3906 non-null   geometry
dtypes: float64(9), geometry(1), object(2)
memory usage: 396.7+ KB


In [14]:
sensor_wk.to_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/sensor_wk.shp')